<a href="https://colab.research.google.com/github/lovepreetmultani/Anti-Money-Laundering/blob/main/Tiger_graph_laundering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyTigerGraph
import pyTigerGraph as tg
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 4.4 MB/s eta 0:00:00


In [4]:
# Connection parameters
hostName = "https://a55fb64527c24bf68095be5e2188185a.i.tgcloud.io"
userName = "user_2"
password = "Algo@123"
graphName="test_graph"
secret="fk0piafuvq7b9f65cv15djb58l52k6bt"
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")


Connected


In [5]:
graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"SHHHH.... Keep this a secret. Here's your token:\n {authToken}")

SHHHH.... Keep this a secret. Here's your token:
 6gbnbdlv9ks5bthqjs0pf7mp62sl7rpg


In [6]:
show_schema = conn.gsql('''LS''')
print(show_schema)

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX Country(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Customer(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Account(PRIMARY_ID id STRING, init_balance DOUBLE, account_type STRING, tx_behavior INT, pagerank FLOAT, label INT, current_balance DOUBLE, min_send_tx DOUBLE, min_recieve_tx DOUBLE, max_send_tx DOUBLE, max_recieve_tx DOUBLE, avg_send_tx DOUBLE, avg_recieve_tx DOUBLE, cnt_recieve_tx INT, cnt_send_tx INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Transaction(PRIMARY_ID id STRING, tx_behavior_id INT, amount DOUBLE, is_fraud BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Alert(PRIMARY_ID id STRING, alert_type STRING, ts INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- UNDIRECTED EDGE BASED_IN(FROM Customer, TO

In [ ]:
results = conn.gsql('''
USE GRAPH test_graph
''')
print(results)

Using graph 'test_graph'


In [7]:
# DEFINE / CREATE ALL EDGES AND VERTICES
results = conn.gsql('''
USE global
CREATE VERTEX Country(PRIMARY_ID id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Customer (PRIMARY_ID id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Account (PRIMARY_ID id STRING, init_balance DOUBLE, account_type STRING, tx_behavior INT, pagerank FLOAT, label INT, current_balance DOUBLE, min_send_tx DOUBLE, min_recieve_tx DOUBLE, max_send_tx DOUBLE, max_recieve_tx DOUBLE, avg_send_tx DOUBLE, avg_recieve_tx DOUBLE, cnt_recieve_tx INT, cnt_send_tx INT) WITH primary_id_as_attribute="true"
CREATE VERTEX Transaction (PRIMARY_ID id STRING, tx_behavior_id INT, amount DOUBLE, is_fraud BOOL) WITH primary_id_as_attribute="true"
CREATE VERTEX Alert (PRIMARY_ID id STRING, alert_type STRING, ts INT) WITH primary_id_as_attribute="true"
CREATE UNDIRECTED EDGE BASED_IN (From Customer, To Country)
CREATE UNDIRECTED EDGE CUSTOMER_ACCOUNT (From Customer, To Account)
CREATE UNDIRECTED EDGE TRANSACTION_FLAGGED (From Transaction, To Alert)
CREATE DIRECTED EDGE SEND_TO (From Account, To Account) WITH REVERSE_EDGE="reverse_SEND_TO"
CREATE DIRECTED EDGE SEND_TRANSACTION (From Account, To Transaction, ts INT, tx_type STRING) WITH REVERSE_EDGE="reverse_SEND_TRANSACTION"
CREATE DIRECTED EDGE RECEIVE_TRANSACTION (From Transaction, To Account, ts INT, tx_type STRING) WITH REVERSE_EDGE="reverse_RECEIVE_TRANSACTION"
''')
print(results)

Semantic Check Fails: The vertex name Country is used by another object! Please use a different name.
Failed to create vertex types: [Country].


In [8]:
results = conn.gsql('CREATE GRAPH test_graph_new(Country, Customer, Account, Transaction, Alert, BASED_IN, CUSTOMER_ACCOUNT, TRANSACTION_FLAGGED, SEND_TO, SEND_TRANSACTION, reverse_SEND_TRANSACTION, RECEIVE_TRANSACTION, reverse_RECEIVE_TRANSACTION)')

In [10]:
conn.graphname="test_graph_new"
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
conn = tg.TigerGraphConnection(host=hostName, graphname="test_graph_new", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

rckre5gv34n70ukuf6tsa4b2qa5lt7ap


In [11]:
show_schema = conn.gsql('''LS''')
print(show_schema)

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX Country(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Customer(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Account(PRIMARY_ID id STRING, init_balance DOUBLE, account_type STRING, tx_behavior INT, pagerank FLOAT, label INT, current_balance DOUBLE, min_send_tx DOUBLE, min_recieve_tx DOUBLE, max_send_tx DOUBLE, max_recieve_tx DOUBLE, avg_send_tx DOUBLE, avg_recieve_tx DOUBLE, cnt_recieve_tx INT, cnt_send_tx INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Transaction(PRIMARY_ID id STRING, tx_behavior_id INT, amount DOUBLE, is_fraud BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX Alert(PRIMARY_ID id STRING, alert_type STRING, ts INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- UNDIRECTED EDGE BASED_IN(FROM Customer, TO

In [ ]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['Country', 'Customer', 'Account', 'Transaction', 'Alert']
Edges: ['BASED_IN', 'CUSTOMER_ACCOUNT', 'TRANSACTION_FLAGGED', 'SEND_TO', 'SEND_TRANSACTION', 'RECEIVE_TRANSACTION']


In [ ]:
import pandas as pd
!git clone https://github.com/TigerGraph-OSS/AMLSim_Python_Lab
accounts = pd.read_csv("AMLSim_Python_Lab/data/accounts.csv", sep=",")
alerts = pd.read_csv("AMLSim_Python_Lab/data/alerts.csv", sep=",")
transactions = pd.read_csv("AMLSim_Python_Lab/data/transactions.csv", sep=",")

Cloning into 'AMLSim_Python_Lab'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 113 (delta 43), reused 66 (delta 18), pack-reused 0
Receiving objects: 100% (113/113), 11.46 MiB | 16.29 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
accounts.head(10)

,ACCOUNT_ID,CUSTOMER_ID,INIT_BALANCE,COUNTRY,ACCOUNT_TYPE,IS_FRAUD,TX_BEHAVIOR_ID
0,0,C_0,184.44,US,I,False,1
1,1,C_1,175.80,US,I,False,1
2,2,C_2,142.06,US,I,False,1
3,3,C_3,125.89,US,I,False,1
4,4,C_4,151.13,US,I,False,1
5,5,C_5,140.49,US,I,False,1
6,6,C_6,178.38,US,I,False,1
7,7,C_7,130.33,US,I,False,1
8,8,C_8,147.66,US,I,False,1
9,9,C_9,158.34,US,I,False,1


In [ ]:
accounts['ACCOUNT_ID'] = accounts['ACCOUNT_ID'].astype(str)
alerts['ALERT_ID'] = alerts['ALERT_ID'].astype(str)
alerts['SENDER_ACCOUNT_ID'] = alerts['SENDER_ACCOUNT_ID'].astype(str)
alerts['RECEIVER_ACCOUNT_ID'] = alerts['RECEIVER_ACCOUNT_ID'].astype(str)
transactions['TX_ID'] = transactions['TX_ID'].astype(str)
transactions['SENDER_ACCOUNT_ID'] = transactions['SENDER_ACCOUNT_ID'].astype(str)
transactions['RECEIVER_ACCOUNT_ID'] = transactions['RECEIVER_ACCOUNT_ID'].astype(str)

In [ ]:
conn.graphname = "test_graph"
conn.apiToken = conn.getToken(conn.createSecret())

In [ ]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("Account"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("send_money"))


Results for Post vertex
{
  "Config": {
    "STATS": "OUTDEGREE_BY_EDGETYPE",
    "PRIMARY_ID_AS_ATTRIBUTE": true
  },
  "Attributes": [
    {
      "AttributeType": {
        "Name": "DOUBLE"
      },
      "AttributeName": "init_balance"
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "AttributeName": "account_type"
    },
    {
      "AttributeType": {
        "Name": "INT"
      },
      "AttributeName": "tx_behavior"
    },
    {
      "AttributeType": {
        "Name": "FLOAT"
      },
      "AttributeName": "pagerank"
    },
    {
      "AttributeType": {
        "Name": "INT"
      },
      "AttributeName": "label"
    },
    {
      "AttributeType": {
        "Name": "DOUBLE"
      },
      "AttributeName": "current_balance"
    },
    {
      "AttributeType": {
        "Name": "DOUBLE"
      },
      "AttributeName": "min_send_tx"
    },
    {
      "AttributeType": {
        "Name": "DOUBLE"
      },
      "AttributeName": "min_recieve_tx"
    }

In [ ]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 1 Country vertices in the graph
There are 10000 Customer vertices in the graph
There are 1323235 Account vertices in the graph
There are 1323234 Transaction vertices in the graph
There are 391 Alert vertices in the graph
--------------
Edge Counts
There are 10000 BASED_IN edges in the graph
There are 10000 CUSTOMER_ACCOUNT edges in the graph
There are 1719 TRANSACTION_FLAGGED edges in the graph
There are 0 SEND_TO edges in the graph
There are 1323234 SEND_TRANSACTION edges in the graph
There are 1323234 RECEIVE_TRANSACTION edges in the graph


In [ ]:
results = conn.getVerticesById("Account", "42")
pprint(results)

[
  {
    "v_id": "42",
    "v_type": "Account",
    "attributes": {
      "id": "42",
      "init_balance": 186.76,
      "account_type": "I",
      "tx_behavior": 1,
      "pagerank": 0,
      "label": 0,
      "current_balance": 0,
      "min_send_tx": 0,
      "min_recieve_tx": 0,
      "max_send_tx": 0,
      "max_recieve_tx": 0,
      "avg_send_tx": 0,
      "avg_recieve_tx": 0,
      "cnt_recieve_tx": 0,
      "cnt_send_tx": 0
    }
  }
]


In [ ]:
df1 = conn.getVertexDataframe("Alert")
print(df1)

    v_id   id alert_type   ts
0    217  217      cycle  173
1    251  251      cycle  188
2     13   13     fan_in  115
3    180  180     fan_in  175
4    183  183     fan_in    7
..   ...  ...        ...  ...
386  321  321      cycle  198
387  176  176     fan_in    8
388  141  141     fan_in   68
389  147  147     fan_in   74
390   56   56     fan_in   99

[391 rows x 4 columns]


In [ ]:
print(conn.gsql('''
USE GRAPH test_graph
CREATE QUERY accountActivity() FOR GRAPH test_graph {
SumAccum<DOUBLE> @s_sumAmt, @r_sumAmt;
SumAccum<DOUBLE> @s_txCnt, @r_txCnt;
MinAccum<DOUBLE> @s_minAmt, @r_minAmt;
MaxAccum<DOUBLE> @s_maxAmt, @r_maxAmt;
AvgAccum @s_avgAmt, @r_avgAmt;
Seed = {Account.*};
acctSend = SELECT tgt FROM Seed:s -(SEND_TRANSACTION:e)-> Transaction:tgt
ACCUM s.@s_sumAmt += tgt.amount, s.@s_txCnt += 1, s.@s_minAmt += tgt.amount, s.@s_maxAmt += tgt.amount, s.@s_avgAmt += tgt.amount
POST-ACCUM s.current_balance = s.@s_sumAmt - s.init_balance, s.min_send_tx = s.@s_minAmt, s.max_send_tx = s.@s_maxAmt, s.avg_send_tx = s.@s_avgAmt, s.cnt_send_tx = s.@s_txCnt;
acctRecieve = SELECT tgt FROM Seed:s -(RECEIVE_TRANSACTION:e)-> Transaction:tgt
ACCUM s.@r_sumAmt += tgt.amount, s.@r_txCnt += 1, s.@r_minAmt += tgt.amount, s.@r_maxAmt += tgt.amount, s.@r_avgAmt += tgt.amount
POST-ACCUM s.current_balance = s.@r_sumAmt + s.init_balance, s.min_recieve_tx = s.@r_minAmt, s.max_recieve_tx = s.@r_maxAmt, s.avg_recieve_tx = s.@r_avgAmt, s.cnt_recieve_tx = s.@r_txCnt;
PRINT "Features Have Been Calculated";
}
INSTALL QUERY accountActivity
''', options=[]))

Using graph 'test_graph'

Type Check Error in query accountActivity (TYP-111): line 13, col 30
In the FROM clause 'from Seed : s -(RECEIVE_TRANSACTION : e)-> Transaction :
tgt', the source vertex types are [Account], the edge types are
[RECEIVE_TRANSACTION]. However, the edge types [RECEIVE_TRANSACTION] cannot find
any compatible starting vertex types from the source.
Saved as draft query with type/semantic error: [accountActivity].


In [ ]:
results = conn.gsql('''
USE GRAPH test_graph
''')
print(results)

Using graph 'test_graph'


In [ ]:
!head -n 2 Anti-Money-Laundering/accounts.csv

head: cannot open 'Anti-Money-Laundering/accounts.csv' for reading: No such file or directory


In [ ]:
results = conn.gsql('''
DROP JOB load_job_accounts
  END
  ''')

In [ ]:
results = conn.gsql('''
  USE GRAPH test_graph_New
  BEGIN
  CREATE LOADING JOB load_job_accounts FOR GRAPH test_graph_New {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Customer VALUES($1) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO VERTEX Account VALUES($0, $2, $4, $6, _, _, _, _, _, _, _, _, _, _, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE BASED_IN VALUES($1, $3) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE CUSTOMER_ACCOUNT VALUES($1, $4) USING SEPARATOR=",", HEADER="true", EOL="\n";
}
  END
  ''')
print(results)

Using graph 'test_graph_New'
Successfully created loading jobs: [load_job_accounts].


In [ ]:
results = conn.gsql('''
USE GRAPH test_graph_New
''')
print(results)

Using graph 'test_graph_New'


In [ ]:
!head -n 2 Anti-Money-Laundering/alerts.csv

head: cannot open 'Anti-Money-Laundering/alerts.csv' for reading: No such file or directory


In [ ]:
results = conn.gsql('''
DROP JOB load_job_alerts
  END
  ''')

In [ ]:
results = conn.gsql('''
  USE GRAPH test_graph_New
  BEGIN
CREATE LOADING JOB load_job_alerts FOR GRAPH test_graph_New {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Alert VALUES($0, $1, $8) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE TRANSACTION_FLAGGED VALUES($3, $0) USING SEPARATOR=",", HEADER="true", EOL="\n";
}
  END
  ''')
print(results)

Using graph 'test_graph_New'
Successfully created loading jobs: [load_job_alerts].


In [ ]:
results = conn.gsql('''
USE GRAPH test_graph_New
''')
print(results)

Using graph 'test_graph_New'


In [ ]:
!head -n 2 Anti-Money-Laundering/transactions.csv

head: cannot open 'Anti-Money-Laundering/transactions.csv' for reading: No such file or directory


In [ ]:
results = conn.gsql('''
DROP JOB load_job_transactions
  END
  ''')

In [ ]:
results = conn.gsql('''
  USE GRAPH test_graph_New
  BEGIN
CREATE LOADING JOB load_job_transactions FOR GRAPH test_graph_New {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Transaction VALUES($0, _, $4, $6) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE SEND_TO VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE SEND_TRANSACTION VALUES($1, $0, $5, $3) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE RECEIVE_TRANSACTION VALUES($0, $2, $5, $3) USING SEPARATOR=",", HEADER="true", EOL="\n";
}
  END
  ''')
print(results)

Using graph 'test_graph_New'
Successfully created loading jobs: [load_job_transactions].
